In [1]:
from keras.models import Sequential
from keras.layers import LSTM, RepeatVector, Dense, Activation
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import numpy as np
from six.moves import range
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 100000
SPLIT_RATIO = 0.8
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
OUTPUTLEN = DIGITS + 1
chars = '0123456789+- '
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

# Data Generation

In [6]:
questions = []
expected = []
seen = set()
symbol = '+'
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    
    if symbol == '-' and a < b:
        a, b = b, a

    seen.add(key)
    q = '{}{}{}'.format(a, symbol, b)
    query = q + ' ' * (MAXLEN - len(q))
    
    if symbol == '+':
        ans = str(a + b)
    else:
        ans = str(a - b)
    
    ans += ' ' * (OUTPUTLEN - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
    
    if symbol == '+':
        symbol = '-'
    else:
        symbol = '+'
    
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 100000


In [7]:
print(questions[:5], expected[:5])

['46+88  ', '841-305', '573+5  ', '720-87 ', '23+718 '] ['134 ', '536 ', '578 ', '633 ', '741 ']


# Processing

In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), OUTPUTLEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, OUTPUTLEN)

Vectorization...


In [9]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

idx = int(SPLIT_RATIO * TRAINING_SIZE)

# train_test_split
train_x = x[:idx]
train_y = y[:idx]
test_x = x[idx:]
test_y = y[idx:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(54000, 7, 13)
(54000, 4, 13)
Validation Data:
(6000, 7, 13)
(6000, 4, 13)
Testing Data:
(40000, 7, 13)
(40000, 4, 13)


In [10]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False  True False False False False
   False False]
  [False False False False False False False False False False False
    True False]
  [False False False False False False False  True False False False
   False False]
  [False  True False False False False False False False False False
   False False]
  [False False False False False False False False False  True False
   False False]
  [False False False False False False  True False False False False
   False False]
  [False False False False False False  True False False False False
   False False]]

 [[False False False False  True False False False False False False
   False False]
  [False False False False False False False False False False False
   False  True]
  [False  True False False False False False False False False False
   False False]
  [False False False False False False False False False False  True
   False False]
  [False False False False False False False False Fals

# Build Model

In [11]:
print('Build model...')

def build_model():
    input_shape = (MAXLEN, len(chars))

    model = Sequential()

    # Encoder:
    model.add(Bidirectional(LSTM(20), input_shape=input_shape))
    model.add(BatchNormalization())

    # The RepeatVector-layer repeats the input n times
    model.add(RepeatVector(OUTPUTLEN))

    # Decoder:
    model.add(Bidirectional(LSTM(20, return_sequences=True)))
    model.add(BatchNormalization())

    model.add(TimeDistributed(Dense(len(chars))))
    model.add(Activation('softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.01),
        metrics=['accuracy'],
    )

    return model

model = build_model()
print(model.summary())

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 40)                5440      
_________________________________________________________________
batch_normalization_1 (Batch (None, 40)                160       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 40)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 4, 40)             9760      
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 40)             160       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             533       
_______________________________________________________

# Training

In [12]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    history = model.fit(x_train, y_train,
                        batch_size=BATCH_SIZE,
                        epochs=1,
                        validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 8s 154us/step - loss: 1.6106 - acc: 0.4218 - val_loss: 1.4652 - val_acc: 0.4596
Q 872-762 T 110  ☒ 24  
Q 535-297 T 238  ☒ 236 
Q 618-581 T 37   ☒ 11  
Q 931+9   T 940  ☒ 915 
Q 7+262   T 269  ☒ 344 
Q 545-40  T 505  ☒ 499 
Q 55+999  T 1054 ☒ 100 
Q 95+546  T 641  ☒ 645 
Q 983+98  T 1081 ☒ 900 
Q 151+195 T 346  ☒ 215 

--------------------------------------------------
Iteration 1
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 6s 104us/step - loss: 1.2701 - acc: 0.5239 - val_loss: 1.2194 - val_acc: 0.5455
Q 987-6   T 981  ☒ 988 
Q 552-84  T 468  ☒ 472 
Q 691+21  T 712  ☒ 713 
Q 337-58  T 279  ☒ 274 
Q 619+682 T 1301 ☒ 1282
Q 638-3   T 63

54000/54000 [==============================] - 6s 103us/step - loss: 0.2041 - acc: 0.9249 - val_loss: 0.1916 - val_acc: 0.9317
Q 89+811  T 900  ☒ 910 
Q 922-80  T 842  ☑ 842 
Q 55-51   T 4    ☑ 4   
Q 790+735 T 1525 ☒ 1535
Q 7+101   T 108  ☑ 108 
Q 739-27  T 712  ☑ 712 
Q 97+528  T 625  ☑ 625 
Q 55+806  T 861  ☒ 851 
Q 690+527 T 1217 ☑ 1217
Q 45+106  T 151  ☒ 141 

--------------------------------------------------
Iteration 14
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 6s 102us/step - loss: 0.1890 - acc: 0.9319 - val_loss: 0.1528 - val_acc: 0.9432
Q 37+20   T 57   ☑ 57  
Q 57+367  T 424  ☑ 424 
Q 43-31   T 12   ☒ 22  
Q 51+178  T 229  ☒ 239 
Q 412-51  T 361  ☑ 361 
Q 508+14  T 522  ☑ 522 
Q 444+891 T 1335 ☑ 1335
Q 52+70   T 122  ☑ 122 
Q 108-0   T 108  ☑ 108 
Q 210-2   T 208  ☑ 208 

--------------------------------------------------
Iteration 15
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [=====

54000/54000 [==============================] - 5s 100us/step - loss: 0.0992 - acc: 0.9683 - val_loss: 0.0637 - val_acc: 0.9790
Q 530-41  T 489  ☑ 489 
Q 726-4   T 722  ☑ 722 
Q 88+283  T 371  ☑ 371 
Q 281-19  T 262  ☑ 262 
Q 542-187 T 355  ☑ 355 
Q 591-26  T 565  ☑ 565 
Q 523-75  T 448  ☑ 448 
Q 516+786 T 1302 ☑ 1302
Q 571-527 T 44   ☒ 54  
Q 614+73  T 687  ☑ 687 

--------------------------------------------------
Iteration 42
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 5s 99us/step - loss: 0.0755 - acc: 0.9754 - val_loss: 0.0551 - val_acc: 0.9836
Q 175+696 T 871  ☒ 881 
Q 504-17  T 487  ☑ 487 
Q 64+484  T 548  ☑ 548 
Q 580+103 T 683  ☑ 683 
Q 594-532 T 62   ☑ 62  
Q 533-136 T 397  ☑ 397 
Q 594-532 T 62   ☑ 62  
Q 1+933   T 934  ☑ 934 
Q 483+201 T 684  ☑ 684 
Q 458-20  T 438  ☑ 438 

--------------------------------------------------
Iteration 43
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [======

54000/54000 [==============================] - 5s 100us/step - loss: 0.0473 - acc: 0.9853 - val_loss: 0.0233 - val_acc: 0.9935
Q 566-3   T 563  ☑ 563 
Q 74+971  T 1045 ☑ 1045
Q 21+939  T 960  ☑ 960 
Q 300-7   T 293  ☑ 293 
Q 705+51  T 756  ☑ 756 
Q 224-0   T 224  ☑ 224 
Q 41+232  T 273  ☑ 273 
Q 85+555  T 640  ☑ 640 
Q 951+524 T 1475 ☑ 1475
Q 276-226 T 50   ☑ 50  

--------------------------------------------------
Iteration 70
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 5s 101us/step - loss: 0.0619 - acc: 0.9811 - val_loss: 0.0642 - val_acc: 0.9825
Q 795-51  T 744  ☑ 744 
Q 88+845  T 933  ☑ 933 
Q 761-77  T 684  ☑ 684 
Q 795-398 T 397  ☑ 397 
Q 927-373 T 554  ☑ 554 
Q 989-28  T 961  ☑ 961 
Q 536-526 T 10   ☑ 10  
Q 864+352 T 1216 ☑ 1216
Q 966+519 T 1485 ☑ 1485
Q 855-316 T 539  ☑ 539 

--------------------------------------------------
Iteration 71
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [=====

54000/54000 [==============================] - 6s 102us/step - loss: 0.0648 - acc: 0.9814 - val_loss: 0.0146 - val_acc: 0.9967
Q 425-28  T 397  ☑ 397 
Q 546-458 T 88   ☑ 88  
Q 898-496 T 402  ☑ 402 
Q 834-2   T 832  ☑ 832 
Q 459+524 T 983  ☑ 983 
Q 743-65  T 678  ☑ 678 
Q 658-53  T 605  ☑ 605 
Q 358-341 T 17   ☑ 17  
Q 147+18  T 165  ☑ 165 
Q 162+556 T 718  ☑ 718 

--------------------------------------------------
Iteration 98
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 5s 101us/step - loss: 0.0433 - acc: 0.9875 - val_loss: 0.0383 - val_acc: 0.9902
Q 401+75  T 476  ☑ 476 
Q 834-62  T 772  ☑ 772 
Q 427-63  T 364  ☑ 364 
Q 8+283   T 291  ☑ 291 
Q 274-10  T 264  ☑ 264 
Q 647+145 T 792  ☑ 792 
Q 76+229  T 305  ☑ 305 
Q 505-402 T 103  ☑ 103 
Q 44+914  T 958  ☑ 958 
Q 769-447 T 322  ☑ 322 

--------------------------------------------------
Iteration 99
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [=====

In [13]:
# history = model.fit(x_train, y_train,
#                     batch_size=BATCH_SIZE,
#                     epochs=100,
#                     validation_data=(x_val, y_val))

# Testing

In [14]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
    

MSG : Prediction


In [15]:
pred = model.predict_classes(test_x)

In [16]:
prediction = np.apply_along_axis(ctable.decode, 1, pred, False)

In [17]:
answer = np.apply_along_axis(lambda x: "".join(ctable.indices_char[i] for i in x), 1, test_y.argmax(axis=-1))

In [18]:
np.sum(prediction == answer) / answer.shape[0]

0.987